<a href="https://colab.research.google.com/github/anon560096/bds_project/blob/master/code/yelp_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from shutil import copyfile
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn import manifold, datasets
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import confusion_matrix
from scipy.spatial.distance import squareform
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.ticker import NullFormatter
from google.colab import files
import PIL
import json

In [0]:
!mkdir -p /content/data
!mkdir -p /content/data/models

In [0]:
# Mount drive for dataset
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# print('Upload tar file for yelp photos')
# os.chdir('/content/data/')
# uploaded = files.upload()
!cp drive/My\ Drive/yelp_photos.tar data/

In [0]:
print('Extracting yelp photos...')
!tar -xvf 'data/yelp_photos.tar' > filenames.txt
print('Done extraction. Deleting the tar file...')
!rm yelp_photos.tar
print('Deleted!')

Extracting yelp photos...
Done extraction. Deleting the tar file...
rm: cannot remove 'yelp_photos.tar': No such file or directory
Deleted!


In [0]:
# print('Upload the trained model \'.pkl\' file')
# os.chdir('/content/data/models/')
# uploaded = files.upload()
!cp drive/My\ Drive/trained_model.pkl data/models/

In [0]:
path = Path('')
os.chdir('/content/')
test_data = ImageList.from_folder(path/'photos')
learnt_model = load_learner(path/'data/models', 'trained_model.pkl', test=test_data)

In [0]:
class_to_index = learnt_model.data.c2i
index_to_class = {}
for key,val in class_to_index.items():
  index_to_class[val] = key

In [0]:
learnt_model = learnt_model.to_fp32()
preds, _ = learnt_model.get_preds(ds_type=DatasetType.Test)
labels = np.argmax(preds, 1)

In [0]:
# print(labels)
# print(index_to_class)
# print(test_data.items)
labels = labels.numpy()

with open('/content/data/labels.json', 'w+') as f:
  for i, img_path in enumerate(test_data.items):
    json_data = {}
    img_name = os.path.basename(os.path.normpath(img_path.stem))
    img_id = os.path.splitext(img_name)[0]
    json_data['id'] = img_id
    json_data['101_label'] = index_to_class[labels[i]]
    if i == 0:
      f.write(json.dumps(json_data))
    else:
      f.write('\n' + json.dumps(json_data))
f.close()
